In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

Missouri
--------
Election results from the Secretary of State's office via OpenElections (https://github.com/openelections/openelections-data-mo/tree/master/2016). Absentee,provisional, and mail ballots were reported countywide in nearly every county; these were distributed by candidate to precincts based on their share of the precinct-level reported vote.

Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release. Monroe County comes from the 2010 Census VTD release. Platte County comes from the 2020 Census VTD release.  The following counties instead used maps or shapefiles sourced from the respective county governments: Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.

Precinct mergers were made in the following counties to match county reporting units: Andrew, Audrain, Barry, Benton, Bollinger, Boone, Callaway, Camden, Cape Girardeau, Carroll, Carter, Cass, Christian, Clark, Cole, Cooper, Crawford, Dallas, Dekalb, Douglas, Dunklin, Franklin, Gentry, Greene, Grundy, Howard, Howell, Jackson, Jasper, Jefferson, Johnson, Linn, Livingston, Marion, Mercer, Mississippi, Moniteau, Morgan, New Madrid, Nodaway, Oregon, Osage, Pemiscot, Pettis, Phelps, Platte, Polk, Putnam, Ralls, Randolph, Ray, Ripley, St. Clair, St. Louis, Schuyler, Shannon, Shelby, Stone, Sullivan, Warren, Wayne, Webster, Worth.

Additional modifications to reflect how election results were reported in 2016:

Audrain: Add Vandalia City; Align Mexico precincts with municipal code
Barry: Add Monett City; Align Cassville with corporate boundary
Barton: Adjust Lamar ward boundaries to match city map
Bollinger: Adjust Patton/Sedgewickville to match county GIS
Butler: Align Poplar Bluff with corporate boundary and adjust wards to match city map
Callaway: Align Fulton, Holts Summit precincts with 2016 corporate boundaries
Carroll: Adjust Carrollton City wards to match voter file
Cass: Align Belton, Pleasant Hill, Raymore precincts with corporate boundaries; Adjust 37 Pleasant Prairie/25 West Peculiar Rural
Chariton: Adjust Salisbury ward boundaries to match city map
Christian: Revise Cassidy 1/Cassidy 2; Reverse Nixa City annexations into Rosedale 1A/1B, Union Chapel 1
Clay: Adjust Excelsior Springs, Kearney, Smithville precincts to match 2016 ward and corporate boundaries; Adjust KC 21-1/15, 21-1/17, 21-3/11, 21-4/6, 21-4/11, 21-5/7/24, 21-6/24, 21-7/20, 21-13/26, 21-14/23, 21-17/16, 21-17/26, 21-17/Platte 1, Gallatin 7/16, 10/13, 10/16, 11/15, 13/15, Kearney 3/Platte 2, Liberty 1/10, 6/13, 7/11, 11/14, Washington 1/3 to match voter file
Clinton: Align Atchison/Concord, Jackson/Lathrop with 2016 precinct split assignments
Cole: Adjust Marion Twp St Martins to include St Martins city in Jefferson Twp
Crawford: Move Benton Twp section of Leasburg precinct from Recklein to Leasburg
Dade: Realign Greenfield wards to match municipal code
Dallas: Split North Benton Inside/OCL; Align N/S Benton with Buffalo ward map
Dent: Split Boss precinct from R-4 precinct; Adjust R-2/R-3 boundary
Dunklin: Align Holcomb, Rives with township boundaries; Adjust Arbyrd/Senath; Align Kennett wards with city map
Franklin: Split Dry Branch between Stanton & St. Clair Out of Town
Gasconade: Align Hermann, Owensville, Rosebud with corporate boundaries and ward descriptions
Howard: Split Old Franklin & New Franklin
Howell: Align West Plains precincts with 2016 corporate boundary
Iron: Adjust Annapolis/Des Arc/Vulcan, Ironton/Pilot Knob
Jasper: Realign Carthage 2/3, Joplin 1/7, 10/13, 10/15, 13/22, Oronogo 1/2, Webb City 1/4 to match city ward maps
Jefferson: Split Festus Ward 4 P1/P2; Adjust Crystal City/Riverview
Johnson: Adjust Warrensburg NE/NW boundary to match county maps
Lewis: Split LaBelle Twp between LaBelle FS/Lewistown FS with PLSS grid
Lincoln: Align Moscow Mills, Troy with 2016 corporate boundaries; Adjust Briscoe/Troy Rural
Linn: Split Brookfield Township between Brookfield 1&2/3&4
Livingston: Align Chillicothe with 2016 corporate boundaries
Macon: Align Macon City wards with corporate boundaries
Madison: Align Fredericktown wards with corporate boundaries
Marion: Split Fabius-Smileyville/Taylor, Warren-Monroe City/Warren; Align Hannibal wards with voter file
Miller: Adjust all precinct boundaries to match voter file
Mississippi: Adjust Bertrand/East Prairie Rural; Align East Prairie City with corporate boundary
Montgomery: Align Bellflower/Middletown, Big Spring/Rhineland with voter file
Morgan: Adjust Versaille North/South boundary to match county GIS
New Madrid: Add New Madrid City wards; Add Big Prairie 4 precinct; Move Portageville Ward 1 to Portage 1
Newton: Align Staples precincts with Joplin City GIS boundaries; Align Neosho precincts with Neosho City GIS boundaries; Adjust Newtonia/Ritchey Wentworth
Pettis: Adjust Sedalia City wards to match city map
Phelps: Align St. James ward boundaries with city map
Pike: Adjust Cuivre JL boundaries with Ashley NO, Buffalo E, Prairieville I
Platte: Align Lake Waukomis, Parkville, Platte City, Riverside/Northmore, Weatherby Lake with corporate boundaries; Adjust Platte Hills/Park Hill, Seven Bridges/Shiloh
Polk: Split Campbell/Jefferson precincts by school district
Pulaski: Adjust Big Piney/St. Roberts, Laquey/Waynesville
Putnam: Split Union Twp into NE/SW precincts to match county map
Ralls: Replace Center/Liberty precincts with 2010 VTDs
Ray: Adjust Richmond City wards, Homestead Village/Wood Heights/Lawson
Reynolds: Realign precincts countywide to match voter file
Saline: Split Nelson precinct from Hardeman precinct; Adjust Marshall Ward 2/3 boundary
St. Charles: Adjust Brian/Ridgepoint, Civic/St. Paul, Harvester/Sycamore, Mamelle/Marina
St. Francois: Align Desloge, Farmington, Park Hills with corporate boundaries and city ward maps; Adjust Bonne Terre/Timberline, Patterson/Timberline, Leadwood/Park Hills to match voter file
St. Louis: Add UNV044 precinct; Revise QUE047 precinct
Scott: Adjust Haywood City/Morley, Sikeston Ward 1/McMullin/Vanduser
Shannon: Align all precincts with county GIS and voter file
Shelby: Adjust Jackson/North River to match voter file
Stoddard: Align Advance, Bernie, Bloomfield, Dexter, Puxico with corporate boundaries; Adjust Advance/Leora, Bernie/Dexter/Pyle, Bloomfield/Gray Ridge
Stone: Move Branson West from Ruth A to Ruth C; Adjust Ruth B City/Rural to match county GIS
Taney: Adjust Boston Center/Walnut Shade; Align Branson wards with 2016 corporate boundary
Vernon: Align Bronaugh/Moundville, Harwood/Schell City, Deerfield/NG Armory with voter file
Washington: Adjust City Hall/Stony Point, Cruise/Richwoods to match county GIS



In [2]:
vest_mo_16 = gp.read_file("./raw-from-source/VEST/mo_2016/mo_2016.shp")

In [3]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Missouri"]
fips_file["FIPS County"]=fips_file["FIPS County"].astype(str)
fips_file["FIPS County"]=fips_file["FIPS County"].str.zfill(3)
fips_file["unique_ID"] =  "42" + fips_file["FIPS County"]
fips_codes = fips_file["unique_ID"].tolist()
mo_fips_dict = dict(zip(fips_file["County Name"],fips_file["FIPS County"]))

In [4]:
elections_2016 = pd.read_csv("./raw-from-source/Open_Elections/openelections-data-mo-master/2016/20161108__mo__general__precinct.csv")

In [5]:
office_list = ['President', 'U.S. Senate', 'Governor', 'Lieutenant Governor','Secretary of State', 'State Treasurer', 'Attorney General']
elections_2016 = elections_2016[elections_2016["office"].isin(office_list)]

In [6]:
candidate_list = ["Donald J. Trump, Michael R. Pence","Roy Blunt","Eric Schmitt","Josh Hawley",
                 "John (Jay) Ashcroft","Eric Greitens","Mike Parson","Hillary Rodham Clinton, Timothy Michael Kaine",
                 "Jason Kander","Chris Koster","Teresa Hensley","Judy Baker","Russ Carnahan","Robin Smith","Chris Morrill",
                 "Gary Johnson, Bill Weld","Jonathan Dine","Steven R. Hedrick","Sean O'Toole","Jennifer Leach","Fred Ryman",
                 "Cisse W Spragins","Johnathan McFarland","Carol Hexem","Lester Benton (Les) Turilli, Jr.","Don Fitz",
                 "Jill Stein, Ajamu Baraka","Darrell L. Castle, Scott N. Bradley"]


elections_2016=elections_2016[elections_2016["candidate"].isin(candidate_list)]

In [7]:
county_name_changes_dict = {"De Kalb":"Dekalb","McDonald":"Mcdonald",
'St. Charles':"St Charles", 
'St. Clair':"St Clair",
'St. Francois':"St Francois",
'St. Louis':"St Louis",
'St. Louis City':"St Louis City",
'Ste. Genevieve':"Ste Genevieve",
"Kansas City":"KSC"}

In [8]:
elections_2016["county"]=elections_2016["county"].map(county_name_changes_dict).fillna(elections_2016["county"])
elections_2016["county"]=elections_2016["county"].map(mo_fips_dict).fillna(elections_2016["county"])

In [9]:
elections_2016["county"].unique()

array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
       '019', '021', '023', '025', '027', '029', '031', '033', '035',
       '037', '039', '041', '043', '045', '047', '049', '051', '053',
       '055', '057', '059', '061', '063', '065', '067', '069', '071',
       '073', '075', '077', '079', '081', '083', '085', '087', '089',
       '091', '093', '095', '097', '099', '101', 'KSC', '103', '105',
       '107', '109', '111', '113', '115', '117', '121', '123', '125',
       '127', '119', '129', '131', '133', '135', '137', '139', '141',
       '143', '145', '147', '149', '151', '153', '155', '157', '159',
       '161', '163', '165', '167', '169', '171', '173', '175', '177',
       '179', '181', '195', '197', '199', '201', '203', '205', '183',
       '185', '187', '189', '510', '186', '207', '209', '211', '213',
       '215', '217', '219', '221', '223', '225', '227', '229'],
      dtype=object)

In [10]:
print(elections_2016.head())
elections_2016["precinct"]=elections_2016["precinct"].str.strip()
elections_2016["pivot_col"]=elections_2016["county"]+elections_2016["precinct"]

  county                precinct     office district party  \
0    001    SOUTHWEST ONE/BENTON  President      NaN   DEM   
1    001    SOUTHEAST TWO/BENTON  President      NaN   DEM   
2    001  SOUTHEAST THREE/BENTON  President      NaN   DEM   
3    001   NORTHEAST FOUR/BENTON  President      NaN   DEM   
4    001   NORTHEAST FIVE/BENTON  President      NaN   DEM   

                                       candidate  votes  
0  Hillary Rodham Clinton, Timothy Michael Kaine    224  
1  Hillary Rodham Clinton, Timothy Michael Kaine    458  
2  Hillary Rodham Clinton, Timothy Michael Kaine    391  
3  Hillary Rodham Clinton, Timothy Michael Kaine    386  
4  Hillary Rodham Clinton, Timothy Michael Kaine    256  


In [11]:
pivoted_2016 = pd.pivot_table(elections_2016,values=["votes"],index=["pivot_col"],columns=["candidate"],aggfunc=sum)
pivoted_2016 = pivoted_2016.fillna(0)

In [12]:
#print(pivoted_2016.head())
pivoted_2016.reset_index(drop=False,inplace=True)
pivoted_2016.columns = pivoted_2016.columns.droplevel(0)

print(pivoted_2016.head())

candidate                            Carol Hexem  Chris Koster  Chris Morrill  \
0                       001ABSENTEE         13.0         570.0           40.0   
1                       001BRASHEAR          4.0         245.0           28.0   
2                        001FEDERAL          0.0           0.0            0.0   
3          001NORTHEAST FIVE/BENTON         13.0         304.0           21.0   
4          001NORTHEAST FOUR/BENTON         20.0         427.0           42.0   

candidate  Cisse W Spragins  Darrell L. Castle, Scott N. Bradley  Don Fitz  \
0                      19.0                                  8.0       3.0   
1                       8.0                                  3.0      13.0   
2                       0.0                                  0.0       0.0   
3                       3.0                                  4.0       5.0   
4                      21.0                                  2.0      13.0   

candidate  Donald J. Trump, Michael R. Pence

G16PRERTRU - Donald J. Trump (Republican Party)
G16PREDCLI - Hillary Rodham Clinton (Democratic Party)
G16PRELJOH - Gary Johnson (Libertarian Party)
G16PREGSTE - Jill Stein (Green Party)
G16PRECCAS - Darrell L. Castle (Constitution Party)

G16USSRBLU - Roy Blunt (Republican Party)
G16USSDKAN - Jason Kander (Democratic Party)
G16USSLDIN - Jonathan Dine (Libertarian Party)
G16USSGMCF - Johnathan McFarland (Green Party)
G16USSCRYM - Fred Ryman (Constitution Party)

G16GOVRGRE - Eric Greitens (Republican Party)
G16GOVDKOS - Chris Koster (Democratic Party)
G16GOVLSPR - Cisse W. Spragins (Libertarian Party)
G16GOVGFIT - Don Fitz (Green Party)
G16GOVITUR - Lester Benton (Les) Turilli, Jr. (Independent)

G16LTGRPAR - Mike Parson (Republican Party)
G16LTGDCAR - Russ Carnahan (Democratic Party)
G16LTGLHED - Steven R. Hedrick (Libertarian Party)
G16LTGGLEA - Jennifer Leach (Green Party)

G16ATGRHAW - Josh Hawley (Republican Party)
G16ATGDHEN - Teresa Hensley (Democratic Party)

G16TRERSCH - Eric Schmitt (Republican Party)
G16TREDBAK - Judy Baker (Democratic Party)
G16TRELOTO - Sean O'Toole (Libertarian Party)
G16TREGHEX - Carol Hexem (Green Party)

G16SOSRASH - John (Jay) Ashcroft (Republican Party)
G16SOSDSMI - Robin Smith (Democratic Party)
G16SOSLMOR - Chris Morrill (Libertarian Party)

In [13]:
pivoted_2016.columns = ["join_col","G16TREGHEX","G16GOVDKOS","G16SOSLMOR","G16GOVLSPR",
                        "G16PRECCAS","G16GOVGFIT",
                        "G16PRERTRU","G16GOVRGRE","G16TRERSCH",
                        "G16USSCRYM","G16PRELJOH",
                        "G16PREDCLI","G16USSDKAN",
                        "G16LTGGLEA","G16PREGSTE","G16SOSRASH"
                        ,"G16USSGMCF","G16USSLDIN","G16ATGRHAW","G16TREDBAK",
                        "G16GOVITUR","G16LTGRPAR","G16SOSDSMI",
                        "G16USSRBLU","G16LTGDCAR"
                        ,"G16TRELOTO","G16LTGLHED","G16ATGDHEN"]

In [14]:
pivoted_2016["STATEFP"]="29"
pivoted_2016["COUNTYFP"]=pivoted_2016["join_col"].str[0:3]
pivoted_2016["NAME"]=pivoted_2016["join_col"].str[3:]

In [15]:
print(pivoted_2016.head())

                   join_col  G16TREGHEX  G16GOVDKOS  G16SOSLMOR  G16GOVLSPR  \
0               001ABSENTEE        13.0       570.0        40.0        19.0   
1               001BRASHEAR         4.0       245.0        28.0         8.0   
2                001FEDERAL         0.0         0.0         0.0         0.0   
3  001NORTHEAST FIVE/BENTON        13.0       304.0        21.0         3.0   
4  001NORTHEAST FOUR/BENTON        20.0       427.0        42.0        21.0   

   G16PRECCAS  G16GOVGFIT  G16PRERTRU  G16GOVRGRE  G16TRERSCH  ...  \
0         8.0         3.0       816.0       762.0       775.0  ...   
1         3.0        13.0       638.0       551.0       610.0  ...   
2         0.0         0.0         3.0         0.0         0.0  ...   
3         4.0         5.0       435.0       428.0       442.0  ...   
4         2.0        13.0       400.0       397.0       387.0  ...   

   G16LTGRPAR  G16SOSDSMI  G16USSRBLU  G16LTGDCAR  G16TRELOTO  G16LTGLHED  \
0       754.0       450.0  

In [16]:
print(vest_mo_16.head())

  STATEFP COUNTYFP                         NAME  G16PRERTRU  G16PREDCLI  \
0      29      189                CHE012/CHE041         480         389   
1      29      043                  Rosedale 1B        1081         360   
2      29      189                        AP028         260         355   
3      29      059                 Benton North         416         114   
4      29      095  KC 2003/2004/2005/2008/2010         875         962   

   G16PRELJOH  G16PREGSTE  G16PRECCAS  G16USSRBLU  G16USSDKAN  ...  \
0          22           8           5         456         429  ...   
1          61          16           8         930         511  ...   
2          24          10           7         199         406  ...   
3          26           5           1         344         184  ...   
4          60          19          18         852        1039  ...   

   G16ATGRHAW  G16ATGDHEN  G16TRERSCH  G16TREDBAK  G16TRELOTO  G16TREGHEX  \
0         542         353         543         330  

In [17]:
print(pivoted_2016.head())

                   join_col  G16TREGHEX  G16GOVDKOS  G16SOSLMOR  G16GOVLSPR  \
0               001ABSENTEE        13.0       570.0        40.0        19.0   
1               001BRASHEAR         4.0       245.0        28.0         8.0   
2                001FEDERAL         0.0         0.0         0.0         0.0   
3  001NORTHEAST FIVE/BENTON        13.0       304.0        21.0         3.0   
4  001NORTHEAST FOUR/BENTON        20.0       427.0        42.0        21.0   

   G16PRECCAS  G16GOVGFIT  G16PRERTRU  G16GOVRGRE  G16TRERSCH  ...  \
0         8.0         3.0       816.0       762.0       775.0  ...   
1         3.0        13.0       638.0       551.0       610.0  ...   
2         0.0         0.0         3.0         0.0         0.0  ...   
3         4.0         5.0       435.0       428.0       442.0  ...   
4         2.0        13.0       400.0       397.0       387.0  ...   

   G16LTGRPAR  G16SOSDSMI  G16USSRBLU  G16LTGDCAR  G16TRELOTO  G16LTGLHED  \
0       754.0       450.0  

In [18]:
vest_mo_16["join_col"]=vest_mo_16['COUNTYFP']+vest_mo_16['NAME']

In [19]:
pivoted_2016 = pivoted_2016[['STATEFP', 'COUNTYFP', 'NAME','G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH',
       'G16PREGSTE', 'G16PRECCAS', 'G16USSRBLU', 'G16USSDKAN', 'G16USSLDIN',
       'G16USSGMCF', 'G16USSCRYM', 'G16GOVRGRE', 'G16GOVDKOS', 'G16GOVLSPR',
       'G16GOVGFIT', 'G16GOVITUR', 'G16LTGRPAR', 'G16LTGDCAR', 'G16LTGLHED',
       'G16LTGGLEA', 'G16ATGRHAW', 'G16ATGDHEN', 'G16TRERSCH', 'G16TREDBAK',
       'G16TRELOTO', 'G16TREGHEX', 'G16SOSRASH', 'G16SOSDSMI', 'G16SOSLMOR',"join_col"]]

In [20]:
print(pivoted_2016.head())

  STATEFP COUNTYFP                   NAME  G16PRERTRU  G16PREDCLI  G16PRELJOH  \
0      29      001               ABSENTEE       816.0       501.0        33.0   
1      29      001               BRASHEAR       638.0       143.0        27.0   
2      29      001                FEDERAL         3.0         5.0         0.0   
3      29      001  NORTHEAST FIVE/BENTON       435.0       256.0        32.0   
4      29      001  NORTHEAST FOUR/BENTON       400.0       386.0        48.0   

   G16PREGSTE  G16PRECCAS  G16USSRBLU  G16USSDKAN  ...  G16ATGRHAW  \
0        10.0         8.0       774.0       543.0  ...       817.0   
1        13.0         3.0       551.0       238.0  ...       633.0   
2         0.0         0.0         1.0         0.0  ...         0.0   
3        11.0         4.0       410.0       311.0  ...       462.0   
4        29.0         2.0       381.0       450.0  ...       423.0   

   G16ATGDHEN  G16TRERSCH  G16TREDBAK  G16TRELOTO  G16TREGHEX  G16SOSRASH  \
0       506.0  

In [21]:
sum_var_1 = 0
sum_var_2 = 0
for i in ['G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH',
       'G16PREGSTE', 'G16PRECCAS', 'G16USSRBLU', 'G16USSDKAN', 'G16USSLDIN',
       'G16USSGMCF', 'G16USSCRYM', 'G16GOVRGRE', 'G16GOVDKOS', 'G16GOVLSPR',
       'G16GOVGFIT', 'G16GOVITUR', 'G16LTGRPAR', 'G16LTGDCAR', 'G16LTGLHED',
       'G16LTGGLEA', 'G16ATGRHAW', 'G16ATGDHEN', 'G16TRERSCH', 'G16TREDBAK',
       'G16TRELOTO', 'G16TREGHEX', 'G16SOSRASH', 'G16SOSDSMI', 'G16SOSLMOR']:
    sum_var_1 += np.sum(vest_mo_16[i])
    sum_var_2 += np.sum(pivoted_2016[i])
print(sum_var_1)
print(sum_var_2)

19418450
19418450.0


In [22]:
print(vest_mo_16.head())

  STATEFP COUNTYFP                         NAME  G16PRERTRU  G16PREDCLI  \
0      29      189                CHE012/CHE041         480         389   
1      29      043                  Rosedale 1B        1081         360   
2      29      189                        AP028         260         355   
3      29      059                 Benton North         416         114   
4      29      095  KC 2003/2004/2005/2008/2010         875         962   

   G16PRELJOH  G16PREGSTE  G16PRECCAS  G16USSRBLU  G16USSDKAN  ...  \
0          22           8           5         456         429  ...   
1          61          16           8         930         511  ...   
2          24          10           7         199         406  ...   
3          26           5           1         344         184  ...   
4          60          19          18         852        1039  ...   

   G16ATGDHEN  G16TRERSCH  G16TREDBAK  G16TRELOTO  G16TREGHEX  G16SOSRASH  \
0         353         543         330           9  

In [23]:


print(pivoted_2016["join_col"].value_counts())
print(vest_mo_16["join_col"].value_counts())

031BYRD 2 PRECINCT            1
213NORTH BRANSON              1
189NRW25                      1
009CASSVILLE RURAL/MINERAL    1
025FAIRVIEW/ DAVIS            1
                             ..
009SELIGMAN                   1
017ZALMA                      1
187DESLOGE 3                  1
223LOWNDES                    1
171LI LINCOLN                 1
Name: join_col, Length: 3237, dtype: int64
189CC203                                   1
047KC 21-4                                 1
189GRA026                                  1
01910/12                                   1
207Bloomfield Ward No. 2                   1
                                          ..
213Merriam Woods                           1
129Medicine/Madison/Washington/Harrison    1
095Sni-A-Bar 49                            1
189FER207                                  1
095Prairie 18                              1
Name: join_col, Length: 3273, dtype: int64


In [24]:
empty_prec_to_create = ['189AP218', '189MR200', '189MHT219', '510STL 12-7', '189BON208', '189CHE212', '189MR209', '189GRA202', '189NW215', '189AP207', '189BON210', '189JEF202', '189QUE209', '189WH203', '189LC200', '189NW213', '189NW203', '189QUE205', '189CC205', '189NRW202', '189NW202', '189LEM208', '189QUE210', '189AP206', '189MER207', '189MR211', '189NRW200', '189AP217', '189MR205', '189MHT214', '189CHE203', '189UNV206', '189WH208', '189NW206', '189CLA203', '189QUE208', '189FLO207', '510STL 25-7', '189NOR208', '189AP230', '189SPL208', '189NRW205', '189FER208', '189AP225', '189LEM202', '510STL 28-3', '189MHT206', '189QUE201', '189CLA206', '189LEM201', '189NW211', '189FLO200', '189QUE202', '189BON204', '189CLA205', '189NW207', '189MHT211', '189NOR204', '189CON202', '189MHT222', '189CON204', '189SPL201', '189NW201', '189CHE214', '189HAD200', '095Van Buren 05', '189NOR222', '189LC203', '189MR204', '189MR207', '189MR212', '189CHE200', '189NOR218', '189AP211', '189NW220', '189CLA200', '189MHT220', '189CC202', '189CON212', '510STL 10-12', '189MHT200', '189NOR202', '189AP215', '189CON213', '189UNV205', '189NW226', '189LC209', '189CLA201', '189CLA202', '189GRA210', '189AP200', '189MID201', '189NW205', '510STL 13-5', '189MER215', '095Prairie 58', '189NOR213', '189MR210', '189BON206', '189FER207', '189MR206', '189AP203', '189AP233', '189CC203', '189QUE203', '189MR213', '189WH200', '189JEF200', '189MHT216', '189MHT208', '189QUE213', '189NW217', '189NRW201', '189NW229', '095Van Buren 06', '189UNV201', '510STL 10-13', '510STL 27-9', '189BON205', '189LAF205', '189GRA206', '189MHT213', '189MR202', '189NOR200', '510STL 14-10', '189TSF201', '189CC221', '189NW216', '189NW219', '189MER214', '189MHT212', '189HAD201', '510STL 10-11', '189LC204', '189MER204', '189LAF200', '095Prairie 63', '189NW210', '189QUE204', '189AP201', '189SPL207', '189MHT203', '189MID202', '189NW209', '189NW225', '189CC208', '189MER202', '189BON202', '189CHE215', '189BON203', '189MHT223', '189NW212', '189CHE204', '189WH202', '189CON205', '189MER200', '189BON200', '189NOR201', '189MER219', '189AP202', '189QUE211', '189NOR203', '189MID204', '189MHT207', '189MER213', '189NW208', '189JEF205', '189UNV208', '189BON207', '189BON211', '189FLO205', '189AP216', '189FLO209', '189MER210', '189NRW203', '189GRA201', '189NW200', '189AP232', '189SF200', '031Scott City', '189FER206', '189AP208', '189GRA209', '189NW214', '189FLO201', '189NW227', '189MER209', '189MID200', '189NW224', '189LEM200', '189AP237', '189NRW204', '189QUE207', '189QUE200', '189LEM205', '189WH201', '189CC214', '189AP214', '189LAF207', '510STL 10-8', '189CON206', '189MR203', '189UNV202', '189NW204', '189CC201', '189CON200', '189LEM204', '189SPL202']
unmatched_empty_vest_precs = vest_mo_16[vest_mo_16["join_col"].isin(empty_prec_to_create)]
unmatched_empty_vest_precs=unmatched_empty_vest_precs[list(pivoted_2016.columns)]
pivoted_2016 = pivoted_2016.append(unmatched_empty_vest_precs)

In [25]:
filter_out_list = ["PROVISIONAL","PROVISIONAL/MILITARY","INTERSTATE","INTERSTATE FORMER RESIDENT","INTRASTATE",
                   "INTRASTATE NEW RESIDENT","FEDERAL","FEDERAL/STATE","ABSENTEE","WRITE-INS",
                  "CENTRAL 1","ABSENTEE INTRASTATE","WRITE IN","CENTRAL POLLING LOCATION","CENTRAL POLL",
                  "MILITARY","MULTI-PRECINCT COLLECTION","DRE","ABSENTEE ","CUMULATIVE","UMOCVA",
                  "NEW RESIDENT","TRANSFER","INTERSTATE01","WRITE INS","INTRASTATE02","INTRASTATE01"]




In [26]:
#189, 510, all seem good






In [32]:
to_dole_out = pivoted_2016[pivoted_2016["NAME"].isin(filter_out_list)]

In [38]:
to_dole_out_totals = pd.DataFrame(to_dole_out.groupby(["COUNTYFP"]).sum())

In [41]:
to_dole_out_totals.reset_index(inplace=True,drop=False)

In [42]:
print(to_dole_out_totals.head())

  COUNTYFP  G16PRERTRU  G16PREDCLI  G16PRELJOH  G16PREGSTE  G16PRECCAS  \
0      001       819.0       506.0        33.0        10.0         8.0   
1      003       434.0       208.0        26.0         6.0         2.0   
2      005       145.0        51.0         9.0         2.0         1.0   
3      007       421.0       246.0        24.0         2.0         3.0   
4      009      1094.0       310.0        28.0         2.0         7.0   

   G16USSRBLU  G16USSDKAN  G16USSLDIN  G16USSGMCF  ...  G16LTGGLEA  \
0       775.0       543.0        20.0         7.0  ...        26.0   
1       387.0       256.0        16.0         5.0  ...        14.0   
2       134.0        54.0         4.0         3.0  ...         4.0   
3       378.0       282.0        20.0        10.0  ...         6.0   
4       969.0       390.0        35.0         9.0  ...        24.0   

   G16ATGRHAW  G16ATGDHEN  G16TRERSCH  G16TREDBAK  G16TRELOTO  G16TREGHEX  \
0       817.0       506.0       775.0       500.0        

In [27]:
source_vest_names = pd.read_csv("./source_vest_names.csv")
print(source_vest_names.head())
source_vest_names_dict = dict(zip(source_vest_names["SOURCE_name"],source_vest_names["VEST_name"]))
pivoted_2016["join_col"]=pivoted_2016["join_col"].map(source_vest_names_dict).fillna(pivoted_2016["join_col"])


               SOURCE_name        VEST_name
0   001RURAL BENTON/BENTON  001Rural Benton
1              001NOVINGER      001Novinger
2  001SOUTHEAST TWO/BENTON   001Southeast 2
3              001BRASHEAR      001Brashear
4  001NORTHEAST SIX/BENTON   001Northeast 6


In [43]:
filtered_election = pivoted_2016[~pivoted_2016["NAME"].isin(filter_out_list)]
precinct_specific_totals = pd.DataFrame(filtered_election.groupby(["COUNTYFP"]).sum())
precinct_specific_totals.reset_index(inplace=True,drop=False)
print(precinct_specific_totals.head())
print(to_dole_out_totals.head())

  COUNTYFP  G16PRERTRU  G16PREDCLI  G16PRELJOH  G16PREGSTE  G16PRECCAS  \
0      001      5211.0      2994.0       393.0       137.0        42.0   
1      003      6231.0      1837.0       310.0        58.0        28.0   
2      005      1915.0       490.0        95.0        12.0         4.0   
3      007      6560.0      2324.0       339.0        71.0        56.0   
4      009     10334.0      2400.0       310.0        64.0        56.0   

   G16USSRBLU  G16USSDKAN  G16USSLDIN  G16USSGMCF  ...  G16LTGGLEA  \
0      4828.0      3655.0       183.0        94.0  ...       213.0   
1      5223.0      2866.0       197.0        78.0  ...       238.0   
2      1727.0       650.0        68.0        16.0  ...        61.0   
3      5348.0      3398.0       292.0       153.0  ...       245.0   
4      9090.0      3474.0       353.0       166.0  ...       325.0   

   G16ATGRHAW  G16ATGDHEN  G16TRERSCH  G16TREDBAK  G16TRELOTO  G16TREGHEX  \
0      5506.0      3128.0      5284.0      3010.0       2

In [52]:
print(len(filtered_election["COUNTYFP"].unique()))
print(len(precinct_specific_totals["COUNTYFP"].unique()))
print(len(to_dole_out_totals["COUNTYFP"].unique()))

116
116
115


In [59]:
for i in list(filtered_election["COUNTYFP"].unique()):
    if i not in (list(to_dole_out_totals["COUNTYFP"].unique())):
        print(i)
        
#Exclude 510 (STL city)

510


In [47]:
print(filtered_election.head())
print(precinct_specific_totals.head())
print(to_dole_out_totals.head())

  STATEFP COUNTYFP                   NAME  G16PRERTRU  G16PREDCLI  G16PRELJOH  \
1      29      001               BRASHEAR       638.0       143.0        27.0   
3      29      001  NORTHEAST FIVE/BENTON       435.0       256.0        32.0   
4      29      001  NORTHEAST FOUR/BENTON       400.0       386.0        48.0   
5      29      001   NORTHEAST SIX/BENTON       556.0       317.0        41.0   
6      29      001               NOVINGER       724.0       179.0        23.0   

   G16PREGSTE  G16PRECCAS  G16USSRBLU  G16USSDKAN  ...  G16ATGRHAW  \
1        13.0         3.0       551.0       238.0  ...       633.0   
3        11.0         4.0       410.0       311.0  ...       462.0   
4        29.0         2.0       381.0       450.0  ...       423.0   
5         8.0         6.0       534.0       383.0  ...       624.0   
6         4.0         4.0       621.0       257.0  ...       705.0   

   G16ATGDHEN  G16TRERSCH  G16TREDBAK  G16TRELOTO  G16TREGHEX  G16SOSRASH  \
1       174.0  

In [66]:
for index, row in filtered_election.iterrows():
    if row["COUNTYFP"]!="510":
    

STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL
STL


In [60]:
for val in filtered_election["join_col"]:
    if val[0:3]!="510":
        

510STL 1-1
510STL 1-2
510STL 1-3
510STL 1-4
510STL 1-5
510STL 1-6
510STL 1-7
510STL 2-1
510STL 2-2
510STL 2-3
510STL 2-4
510STL 2-5
510STL 2-6
510STL 2-7
510STL 2-8
510STL 3-1
510STL 3-2
510STL 3-3
510STL 3-4
510STL 3-5
510STL 3-6
510STL 3-7
510STL 3-8
510STL 3-9
510STL 4-1
510STL 4-2
510STL 4-3
510STL 4-4
510STL 4-5
510STL 4-6
510STL 4-7
510STL 4-8
510STL 4-9
510STL 4-10
510STL 5-1
510STL 5-2
510STL 5-3
510STL 5-4
510STL 5-5
510STL 5-6
510STL 5-7
510STL 5-8
510STL 6-1
510STL 6-2
510STL 6-3
510STL 6-4
510STL 6-5
510STL 6-6
510STL 6-7
510STL 6-8
510STL 6-9
510STL 7-1
510STL 7-2
510STL 7-3
510STL 7-4
510STL 7-5
510STL 7-6
510STL 8-1
510STL 8-2
510STL 8-3
510STL 8-4
510STL 8-5
510STL 8-6
510STL 8-7
510STL 8-8
510STL 8-9
510STL 9-1
510STL 9-2
510STL 9-3
510STL 9-4
510STL 9-5
510STL 9-6
510STL 9-7
510STL 9-8
510STL 9-9
510STL 10-1
510STL 10-2
510STL 10-3
510STL 10-4
510STL 10-5
510STL 10-6
510STL 10-7
510STL 10-9
510STL 10-10
510STL 11-1
510STL 11-2
510STL 11-3
510STL 11-4
510STL 11-5
510ST

In [69]:
races=['G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH',
       'G16PREGSTE', 'G16PRECCAS', 'G16USSRBLU', 'G16USSDKAN', 'G16USSLDIN',
       'G16USSGMCF', 'G16USSCRYM', 'G16GOVRGRE', 'G16GOVDKOS', 'G16GOVLSPR',
       'G16GOVGFIT', 'G16GOVITUR', 'G16LTGRPAR', 'G16LTGDCAR', 'G16LTGLHED',
       'G16LTGGLEA', 'G16ATGRHAW', 'G16ATGDHEN', 'G16TRERSCH', 'G16TREDBAK',
       'G16TRELOTO', 'G16TREGHEX', 'G16SOSRASH', 'G16SOSDSMI', 'G16SOSLMOR']

for race in races:
    add_var = race+"_add"
    rem_var = race+"_rem"
    filtered_election.loc[:,add_var]=0
    filtered_election.loc[:,rem_var]=0

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [70]:
print(filtered_election.head())

  STATEFP COUNTYFP                   NAME  G16PRERTRU  G16PREDCLI  G16PRELJOH  \
1      29      001               BRASHEAR       638.0       143.0        27.0   
3      29      001  NORTHEAST FIVE/BENTON       435.0       256.0        32.0   
4      29      001  NORTHEAST FOUR/BENTON       400.0       386.0        48.0   
5      29      001   NORTHEAST SIX/BENTON       556.0       317.0        41.0   
6      29      001               NOVINGER       724.0       179.0        23.0   

   G16PREGSTE  G16PRECCAS  G16USSRBLU  G16USSDKAN  ...  G16TRELOTO_add  \
1        13.0         3.0       551.0       238.0  ...               0   
3        11.0         4.0       410.0       311.0  ...               0   
4        29.0         2.0       381.0       450.0  ...               0   
5         8.0         6.0       534.0       383.0  ...               0   
6         4.0         4.0       621.0       257.0  ...               0   

   G16TRELOTO_rem  G16TREGHEX_add  G16TREGHEX_rem  G16SOSRASH_add  \

In [ ]:
for race in races:
    #Create two new variables
    add_var = race+"_add"
    rem_var = race+"_rem"
    #Add_var is the number of votes to add based on the percentage of total votes
    allegheny[add_var]=allegheny[race].apply(lambda x:(x/int(allegheny_sum.loc[race]))*int(allegheny_totals.loc[race]))
    #Rem_var is the decimal number of votes of Add_var
    allegheny[rem_var]=allegheny[add_var]%1
    #Because we don't want to assign 
    to_go = int(np.round((sum(allegheny[add_var]))-sum(np.floor(allegheny[add_var])),0))
    for i in allegheny[rem_var].nlargest(to_go).index:
        allegheny.loc[i,add_var]=np.ceil(allegheny.loc[i,add_var])
    allegheny[add_var]=np.floor(allegheny[add_var])
    allegheny[race]+=allegheny[add_var]
    allegheny[race]=allegheny[race].astype(int)

In [30]:
join_attempt_one = pd.merge(vest_mo_16,filtered_election,how="outer",on="join_col",indicator=True)


In [37]:
print(join_attempt_one["_merge"].value_counts())

left_only = join_attempt_one[join_attempt_one["_merge"]=="left_only"]
right_only = join_attempt_one[join_attempt_one["_merge"]=="right_only"]
both = join_attempt_one[join_attempt_one["_merge"]=="both"]

left_only.to_csv("./exported_csvs/only_vest.csv")
right_only.to_csv("./exported_csvs/only_source.csv")


both.to_csv("./exported_csvs/both.csv")


both          3273
left_only        0
right_only       0
Name: _merge, dtype: int64
